In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [3]:
records = pd.read_csv("/Users/anoutsala/Documents/datathon_2024/records.csv")

In [4]:
records.head()

,event_date,month,year,event_type,sub_event_type,actor1,inter1,inter2,interaction,civilian_targeting,region,country,location,latitude,longitude,source_scale,fatalities,year_month,fatalities_binned
0,2024-11-08,11,2024,Violence against civilians,Attack,JNIM: Group for Support of Islam and Muslims,2,7,27,1,Western Africa,Burkina Faso,Bane,11.5756,-0.3566,New media,2,2024-11,1-10
1,2024-11-08,11,2024,Strategic developments,Other,JNIM: Group for Support of Islam and Muslims,2,1,12,0,Western Africa,Burkina Faso,Silmangue,13.8886,-0.6095,Local partner-Other,0,2024-11,0
2,2024-11-08,11,2024,Battles,Armed clash,JNIM: Group for Support of Islam and Muslims,2,4,24,0,Western Africa,Burkina Faso,Gorgadji,14.0329,-0.5197,Local partner-Other,0,2024-11,0
3,2024-11-08,11,2024,Riots,Mob violence,Rioters (Bolivia),5,7,57,1,South America,Bolivia,La Paz,-16.4957,-68.1336,Subnational-National,0,2024-11,0
4,2024-11-08,11,2024,Protests,Peaceful protest,Protesters (Bolivia),6,0,60,0,South America,Bolivia,La Paz,-16.4957,-68.1336,National,0,2024-11,0


In [5]:
conflict_data = records.groupby(['country', 'year', 'month']).size().reset_index(name='event_count')

conflict_data['year_month'] = conflict_data['year'].astype(str) + '-' + conflict_data['month'].astype(str).str.zfill(2)

fig = px.choropleth(conflict_data,
                    locations='country',
                    locationmode='country names',
                    color='event_count',
                    hover_name='country',
                    animation_frame='year_month',  
                    color_continuous_scale='Reds',
                    title="Monthly Conflict Events by Country and Year")

fig.update_layout(
    width=1400,  
    height=800,  
    title={
        'text': "Monthly Conflict Events by Country and Year",
        'x': 0.5,
        'xanchor': 'center'
    },
    geo=dict(
        showframe=True,
        showcoastlines=True,
        projection_type='equirectangular',
    )
)

fig.write_html("/Users/anoutsala/Documents/datathon_2024/Conflict-Mapping-and-Humanitarian-Response-Optimization/conflict_visualizations/monthly_conflict_events.html")

In [8]:
lat_interval = 0.5  
lon_interval = 0.5  


records['lat_bin'] = (records['latitude'] // lat_interval) * lat_interval
records['lon_bin'] = (records['longitude'] // lon_interval) * lon_interval


records['grid_cell'] = records['lat_bin'].astype(str) + ',' + records['lon_bin'].astype(str)


records['year_month'] = pd.to_datetime(
    records[['year', 'month']].assign(day=1)  
)

records = records.sort_values(by='year_month')


records['year_month_str'] = records['year_month'].dt.strftime('%Y-%m')

conflict_grid_time = records.groupby(['lat_bin', 'lon_bin', 'year_month_str']).size().reset_index(name='conflict_count')


conflict_grid_time['lat_center'] = conflict_grid_time['lat_bin'] + lat_interval / 2
conflict_grid_time['lon_center'] = conflict_grid_time['lon_bin'] + lon_interval / 2


conflict_grid_time = conflict_grid_time.sort_values(by='year_month_str')


fig = px.density_mapbox(conflict_grid_time,
                        lat='lat_center',
                        lon='lon_center',
                        z='conflict_count',
                        radius=10,
                        animation_frame='year_month_str',  
                        center={"lat": 0, "lon": 0},
                        zoom=1,
                        mapbox_style="carto-positron",
                        title="Conflict Density Over Time")
fig.update_layout(
    coloraxis_colorbar=dict(title="Number of Conflicts"),
    width=1200,
    height=800
)

fig.write_html("/Users/anoutsala/Documents/datathon_2024/Conflict-Mapping-and-Humanitarian-Response-Optimization/conflict_visualizations/conflict_density.html")

In [13]:
fatalities_data = records.groupby(['country', 'year', 'month'])['fatalities'].sum().reset_index(name='fatalities')

fatalities_data['year_month'] = fatalities_data['year'].astype(str) + '-' + fatalities_data['month'].astype(str).str.zfill(2)

fig_fatalities = px.choropleth(
    fatalities_data,
    locations='country',
    locationmode='country names',
    color='fatalities',
    hover_name='country',
    animation_frame='year_month',  
    color_continuous_scale='Reds',
    title="Monthly Fatalities by Country and Year"
)

fig_fatalities.update_layout(
    width=1400,  
    height=800,  
    title={
        'text': "Monthly Fatalities by Country and Year",
        'x': 0.5,
        'xanchor': 'center'
    },
    geo=dict(
        showframe=True,
        showcoastlines=True,
        projection_type='equirectangular',
    )
)

fig.write_html("/Users/anoutsala/Documents/datathon_2024/Conflict-Mapping-and-Humanitarian-Response-Optimization/conflict_visualizations/monthly_fatalities.html")

In [14]:
records['lat_bin'] = (records['latitude'] // lat_interval) * lat_interval
records['lon_bin'] = (records['longitude'] // lon_interval) * lon_interval

records['grid_cell'] = records['lat_bin'].astype(str) + ',' + records['lon_bin'].astype(str)


records['year_month'] = pd.to_datetime(records[['year', 'month']].assign(day=1))  
records['year_month_str'] = records['year_month'].dt.strftime('%Y-%m')


fatalities_grid_time = records.groupby(['lat_bin', 'lon_bin', 'year_month_str'])['fatalities'].sum().reset_index(name='fatality_count')


fatalities_grid_time['lat_center'] = fatalities_grid_time['lat_bin'] + lat_interval / 2
fatalities_grid_time['lon_center'] = fatalities_grid_time['lon_bin'] + lon_interval / 2


fatalities_grid_time = fatalities_grid_time.sort_values(by='year_month_str')


fig_fatalities_density = px.density_mapbox(
    fatalities_grid_time,
    lat='lat_center',
    lon='lon_center',
    z='fatality_count',
    radius=10,
    animation_frame='year_month_str',  
    center={"lat": 0, "lon": 0},
    zoom=1,
    mapbox_style="carto-positron",
    title="Fatality Density Over Time"
)

fig_fatalities_density.update_layout(
    coloraxis_colorbar=dict(title="Number of Fatalities"),
    width=1200,
    height=800
)

fig.write_html("/Users/anoutsala/Documents/datathon_2024/Conflict-Mapping-and-Humanitarian-Response-Optimization/conflict_visualizations/fatality_density.html")

In [15]:
conflict_data['event_count_pct_change'] = conflict_data.groupby('country')['event_count'].pct_change()


fatalities_data = records.groupby(['country', 'year', 'month'])['fatalities'].sum().reset_index(name='fatalities')


fatalities_data['year_month'] = fatalities_data['year'].astype(str) + '-' + fatalities_data['month'].astype(str).str.zfill(2)


fatalities_data['fatalities_pct_change'] = fatalities_data.groupby('country')['fatalities'].pct_change() * 100


merged_data = conflict_data.merge(fatalities_data[['country', 'year_month', 'fatalities', 'fatalities_pct_change']], 
                                   on=['country', 'year_month'], how='left')


conflict_min, conflict_max = merged_data['event_count_pct_change'].min(), merged_data['event_count_pct_change'].max()
fatalities_min, fatalities_max = merged_data['fatalities_pct_change'].min(), merged_data['fatalities_pct_change'].max()

In [16]:
fig_conflict_change = px.choropleth(merged_data,
                                    locations='country',
                                    locationmode='country names',
                                    color='event_count_pct_change',
                                    hover_name='country',
                                    animation_frame='year_month',  
                                    color_continuous_scale=['blue', 'white', 'red'],
                                    title="Percentage Change in Monthly Conflict Events by Country and Year")

fig_conflict_change.update_layout(
    width=1400,
    height=800,
    title={
        'text': "Percentage Change in Monthly Conflict Events by Country and Year",
        'x': 0.5,
        'xanchor': 'center'
    },
    geo=dict(
        showframe=True,
        showcoastlines=True,
        projection_type='equirectangular',
    )
)

fig.write_html("/Users/anoutsala/Documents/datathon_2024/Conflict-Mapping-and-Humanitarian-Response-Optimization/conflict_visualizations/percent_change_events.html")

In [17]:
fig_fatalities_change = px.choropleth(merged_data,
                                      locations='country',
                                      locationmode='country names',
                                      color='fatalities_pct_change',
                                      hover_name='country',
                                      animation_frame='year_month', 
                                      color_continuous_scale=['blue', 'white', 'red'],
                                      title="Percentage Change in Monthly Fatalities by Country and Year")

fig_fatalities_change.update_layout(
    width=1400,
    height=800,
    title={
        'text': "Percentage Change in Monthly Fatalities by Country and Year",
        'x': 0.5,
        'xanchor': 'center'
    },
    geo=dict(
        showframe=True,
        showcoastlines=True,
        projection_type='equirectangular',
    )
)

fig.write_html("/Users/anoutsala/Documents/datathon_2024/Conflict-Mapping-and-Humanitarian-Response-Optimization/conflict_visualizations/percent_change_fatalities.html")